# Dashboard inflow Canestrari

In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import os

In [6]:
inflow_can = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Dashboard inflow canali e prodotti\Data export\inflow_canestrari.xlsx"

In [7]:
### Variabili
d_parser = lambda x: dt.datetime.strptime(x, '%m.%Y')
columns = ["Anno/Mese Inseriment | Anno Inserimento Ord","Ordini AP","Ordini AC","Budget AP",
           "Budget AC","Sales Director","TIPO PTR","Forecast","Sales Director infloW","linea aggregata"]


### ETL "Inflow Roberta Canestrari"
inflow = pd.read_excel(inflow_can, sheet_name="inflow", engine="openpyxl", header=3, parse_dates=["Anno/Mese Inseriment | Anno Inserimento Ord"], date_parser=d_parser, usecols=columns)


In [8]:
inflow.columns = ["Anno/Mese", "Ordini AP", "Ordini AC", "Budget AP", "Budget AC", "Sales Director", "Rete", "Previsioni impegno", "Area territoriale", "Linea aggregata"]

In [9]:
inflow.fillna({"Ordini AP":0,"Ordini AC":0,"Budget AC":0,"Budget AC":0,"Previsioni impegno":0,"Sales Director":"Missing","Rete": "Misssing","Area territoriale": "Misssing","Linea aggregata": "Misssing"}, inplace=True)

In [10]:
inflow["Ordini"] = inflow.loc[:,['Ordini AP','Ordini AC']].sum(axis=1, min_count=1)
inflow["Budget"]= inflow.loc[:,['Budget AP','Budget AC']].sum(axis=1, min_count=1)

In [11]:
inflow = inflow.loc[:,["Anno/Mese", "Sales Director", "Rete", "Area territoriale", "Linea aggregata", "Ordini", "Budget", "Previsioni impegno"]]

In [12]:
inflow.replace({"Area territoriale":{"Indirect Sales":"Indirect", "North-East_Ibisoft":"North-East Ibisoft"}, "Linea aggregata":{"services":"Services", "hw":"HW"}}, inplace=True)

In [9]:
inflow.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Dashboard inflow canali e prodotti\Source\inflow.xlsx", index=False)

In [13]:
gg_lav = pd.DataFrame(data=pd.date_range(start='2021-01-01', end=(dt.date.today()-pd.Timedelta(days=1))), columns=["Giorno"])

In [18]:
holidays=["2021-01-01","2021-01-06","2021-04-05","2021-05-01","2021-06-02","2021-08-15","2021-11-01","2021-12-08","2021-12-25","2021-12-26"]

gg_lav['N. gg lavorativi'] = np.busday_count(gg_lav["Giorno"].apply(lambda x: (pd.to_datetime(dt.date(x.year,x.month,1))) ).values.astype('datetime64[D]'), 
                                             (gg_lav['Giorno']+pd.Timedelta(days=1)).values.astype('datetime64[D]'), 
                                             weekmask=[1,1,1,1,1,0,0], holidays=holidays)

In [19]:
gg_lav['N. gg lavorativi nel mese'] = gg_lav.groupby(gg_lav["Giorno"].dt.month)["N. gg lavorativi"].transform("max")

In [142]:
gg_lav.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Parametriche\gg_lav.xlsx", index=False)

---

# Dashboard inflow

## Budest

In [17]:
import pandas as pd
import datetime as dt
import numpy as np
import calendar
import zipfile

zip_file =r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Data export\ZQ_MC_BUDANAL_IMP_ALL.zip"
directory_to_extract_to = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Data export"
try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)
except:
    print("Invalid file")

bw_file = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Data export\ZQ_MC_BUDANAL_IMP_ALL_00000.xls"
df = pd.DataFrame(pd.read_html(str(bw_file), encoding = 'utf-8', decimal=",", thousands='.')[3])
df2 = pd.DataFrame(data=df.loc[1:,:])
df2.columns = df.loc[0,:].to_list()
df2["Data Ins. Ordine (monitoraggio)"] = pd.to_datetime(df2["Data Ins. Ordine (monitoraggio)"], format="%d%m%Y")
df2[["Raccolto", "Budget assegnato"]] = df2.loc[:,["Raccolto", "Budget assegnato"]].astype(float)
df2.loc[(df2["Budget assegnato"].notna()) & (df2["Linea Prodotti"] == "Software"), ["Gruppo Prodotti Ricl"]] = "Software"
df2.loc[(df2["Budget assegnato"].notna()) & (df2["Linea Prodotti"] == "Services"), ["Gruppo Prodotti Ricl"]] = "Vendite Services"
df2.rename(columns={'Budget assegnato': 'Budget  assegnato'}, inplace=True)

#df2.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti\inflow_prodotti_nuove_vendite_2022.xlsx", sheet_name='Foglio1', index=False)

df2_curr_month = df2.loc[df2["Data Ins. Ordine (monitoraggio)"].dt.date >= dt.date(dt.datetime.now().year,dt.datetime.now().month,1),:]
#df2_curr_month.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\po_giornaliero\inflow_prodotti_nuove_vendite_2022.xlsx", sheet_name='Foglio1', index=False)


gg_lav = pd.DataFrame(data=pd.date_range(start='2021-01-01', end=(dt.date.today()-pd.Timedelta(days=1))), columns=["Giorno"])

holidays=["2021-01-01","2021-01-06","2021-04-05","2021-05-01","2021-06-02","2021-08-15","2021-11-01","2021-12-08","2021-12-25","2021-12-26"]

gg_lav['N. gg lavorativi'] = np.busday_count(
    gg_lav["Giorno"].apply(lambda x: (pd.to_datetime(dt.date(x.year,x.month,1))) ).values.astype('datetime64[D]'),
    (gg_lav['Giorno']+pd.Timedelta(days=1)).values.astype('datetime64[D]'),
    weekmask=[1,1,1,1,1,0,0], holidays=holidays)

gg_lav['N. gg lavorativi nel mese'] = np.busday_count(
    gg_lav["Giorno"].apply(lambda x: (pd.to_datetime(dt.date(x.year,x.month,1))) ).values.astype('datetime64[D]'),
    (gg_lav['Giorno'].apply(lambda d: dt.date(d.year, d.month, calendar.monthrange(d.year, d.month)[-1]))).values.astype('datetime64[D]'),
    weekmask=[1,1,1,1,1,0,0], holidays=holidays)

#gg_lav.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\Giorni_lavorativi.xlsx", index=False)


---

## PO ordini

In [16]:
import pandas as pd
import datetime as dt
import numpy as np
import zipfile
import shutil


ins_man = shutil.copy(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti\inflow_prodotti_ins_man.xlsx", 
            r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\po_giornaliero\\")

zip_file =r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Data export\ZQ_PO_TAX_GIOR.zip"
directory_to_extract_to = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Data export"
try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)
except:
    print("Invalid file")



bw_file = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Data export\ZQ_PO_TAX_GIOR_00000.xls"
df = pd.DataFrame(pd.read_html(str(bw_file), encoding = 'utf-8', decimal=",", thousands='.')[3])
po = pd.DataFrame(data=df.loc[1:,:])
po.columns = df.loc[0,:].to_list()


po.rename(columns={"Data Ins. Ordine":"Data Ins. Ordine (monitoraggio)","Imponibile Ordine":"Raccolto"}, inplace=True)
po["Budget  assegnato"] = np.nan
po["Data Ins. Ordine (monitoraggio)"] = pd.to_datetime(po["Data Ins. Ordine (monitoraggio)"], format="%d%m%Y")
po[["Raccolto", "Budget  assegnato"]] = po.loc[:,["Raccolto", "Budget  assegnato"]].astype(float)

po.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\po_giornaliero\inflow_prodotti_po_giornaliero.xlsx", sheet_name='Foglio1', index=False)

---